In [23]:
import pandas as pd
import boto3
import json
import configparser
from botocore.exceptions import ClientError
import psycopg2

### Configuration Variables

In [25]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY = config.get('AWS','KEY')
SECRET = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH","DWH_DB")
DWH_DB_USER = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


## Instantiating AWS IAM and Redshift Clients

In [26]:
iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-east-1'
                  )

redshift = boto3.client('redshift',
                       region_name="us-east-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

## Creating New IAM Role

In [5]:
try:
    print("Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )   
    print(f"Role creation successful. Role name: {RoleName}")
except Exception as e:
    print(e)

    
    
print("Attaching S3 Read-Only Policy")
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(f"IAM Role ARN:{roleArn}")

Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.
Attaching S3 Read-Only Policy
IAM Role ARN:arn:aws:iam::156003263193:role/dwhRole


## Creating Redshift Cluster

In [11]:
try:
    response = redshift.create_cluster(        
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

#### Cluster Metadata

In [28]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.ch7ykomwyjzd.us-east-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-00d5510077f546487
7,NumberOfNodes,4


#### Cluster Endpoint & Role ARN

In [16]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.ch7ykomwyjzd.us-east-1.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::156003263193:role/dwhRole


## Create Tables Script

In [1]:
! python create_tables.py

## ETL Script

In [2]:
! python etl.py

## Sample Queries

In [20]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))

#### What are the top 10 most streamed songs? 

In [6]:
cur = conn.cursor()
cur.execute("""
SELECT songs.title, count(*) as streams
FROM songplays 
JOIN songs ON (songplays.song_id = songs.song_id)
GROUP BY songs.title
ORDER BY streams desc
LIMIT 10
""")
cur.fetchall()

[("You're The One", 37),
 ("I CAN'T GET STARTED", 9),
 ('Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 9),
 ("Nothin' On You [feat. Bruno Mars] (Album Version)", 8),
 ("Hey Daddy (Daddy's Home)", 6),
 ('Make Her Say', 5),
 ('Up Up & Away', 5),
 ('Unwell (Album Version)', 4),
 ('Mr. Jones', 4),
 ('Supermassive Black Hole (Album Version)', 4)]

#### Rank the highest usage time of day by hour for song streaming.

In [7]:
cur = conn.cursor()
cur.execute("""
SELECT time.hour, count(*) as streams
FROM songplays 
JOIN time ON (songplays.start_time = time.start_time)
GROUP BY time.hour
ORDER BY streams desc
""")
cur.fetchall()

[(17, 40),
 (18, 26),
 (15, 25),
 (16, 24),
 (8, 18),
 (20, 18),
 (19, 16),
 (14, 16),
 (11, 16),
 (13, 14),
 (7, 13),
 (12, 12),
 (21, 12),
 (23, 11),
 (10, 11),
 (1, 11),
 (9, 9),
 (6, 9),
 (4, 7),
 (5, 7),
 (22, 7),
 (0, 6),
 (2, 3),
 (3, 2)]

#### Who are the top 10 most popular artists?

In [21]:
cur = conn.cursor()
cur.execute("""
SELECT artists.name, count(*) as streams
FROM songplays 
JOIN artists ON (songplays.artist_id = artists.artist_id)
GROUP BY artists.name
ORDER BY streams desc
LIMIT 10
""")
cur.fetchall()

[('Dwight Yoakam', 37),
 ('Kid Cudi / Kanye West / Common', 10),
 ('Kid Cudi', 10),
 ('Ron Carter', 9),
 ('Lonnie Gordon', 9),
 ('B.o.B', 8),
 ('Usher', 6),
 ('Muse', 6),
 ('Usher featuring Jermaine Dupri', 6),
 ('Richard Hawley And Death Ramps_ Arctic Monkeys', 5)]

## Deleting Cluster

In [29]:
try:
    redshift.delete_cluster(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
                            SkipFinalClusterSnapshot=True)
    print("Cluster successfully deleted.")
except Exception as e:
    print("Failure in deleting cluster.")
    print(e)
    
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

Failure in deleting cluster.
An error occurred (InvalidClusterState) when calling the DeleteCluster operation: There is an operation running on the Cluster. Please try to delete it at a later time.


,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,deleting
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.ch7ykomwyjzd.us-east-1.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-00d5510077f546487
7,NumberOfNodes,4
